In [1]:
import pandas as pd
import numpy as np
import re

In [13]:
# Define the log file path
log_file = r"C:\Users\janbe\Documents\GitHub Repos\pulse\output_to_keep\results_benchmark\llms\grok4\zhu_2024b_zero_shot_preprocessor\log_20250721_112523_sepsis_miiv.log"
output_path = r"C:\Users\janbe\Documents\GitHub Repos\pulse\output_to_keep\results_benchmark\llms\grok4\zhu_2024b_zero_shot_preprocessor\Grok4_sepsis_miiv_20250721_112523_metadata_reconstructed.csv"

In [14]:
# Prepare regex patterns for each field
block_start = re.compile(r"Decoded output:", re.MULTILINE)
# Fixed: Since we split on "Decoded output:", the blocks start directly with JSON
decoded_output = re.compile(r'^\s*(\{.*?\})\s*(?=\d{4}-\d{2}-\d{2})', re.DOTALL)
tokenization = re.compile(
    r"Inference time: ([\d.]+)s \| Input Tokens: (\d+) \| Output Tokens: (\d+) \| Thinking Budget: (\d+)"
)
prob_label = re.compile(r"Predicted probability: ([\d.]+) \| True label: (\d+)")
diagnosis = re.compile(r"Diagnosis for: (.+)")
explanation = re.compile(r"Generated explanation: (.+)")

# Read the log file
with open(log_file, "r", encoding="utf-8") as f:
    log = f.read()


# Only process after the start line

start_line = "Starting test evaluation..."

start_idx = log.find(start_line)

if start_idx != -1:
    log = log[start_idx + len(start_line) :]

In [15]:
# Split into blocks
blocks = re.split(block_start, log)
blocks = blocks[1:]  # Skip the first empty block
rows = []

In [16]:
rows = []
for i, block in enumerate(blocks):
    if not block.strip():
        continue

    # Extract fields
    decoded = decoded_output.search(block)
    diagnosis_json = None
    
    # Debug for first few blocks
    if i < 3:
        print(f"Block {i} preview: {block[:100]}...")
        print(f"Decoded match: {decoded is not None}")
    
    if decoded:
        import json
        if i < 3:
            print(f"JSON string: {decoded.group(1)[:100]}...")
        try:
            diagnosis_json = json.loads(decoded.group(1))
            if i < 3:
                print(f"Parsed JSON: {diagnosis_json}")
        except Exception as e:
            if i < 3:
                print(f"JSON parsing error: {e}")
            diagnosis_json = {}

    token_match = tokenization.search(block)
    prob_label_match = prob_label.search(block)
    diagnosis_match = diagnosis.search(block)
    explanation_match = explanation.search(block)

    row = {
        "Input Prompt": "",
        "Target Label": prob_label_match.group(2) if prob_label_match else "",
        "Predicted Probability": prob_label_match.group(1) if prob_label_match else "",
        "Predicted Diagnosis": (
            diagnosis_json.get("diagnosis", "") if diagnosis_json else ""
        ),
        "Predicted Explanation": (
            diagnosis_json.get("explanation", "") if diagnosis_json else ""
        ),
        "Tokenization Time": (
            token_match.group(1) if token_match else ""
        ),
        "Inference Time": (
            token_match.group(1) if token_match else ""
        ),
        "Input Tokens": (
            token_match.group(2) if token_match else ""
        ),
        "Output Tokens": (
            token_match.group(3) if token_match else ""
        ),
    }
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(
    rows,
    columns=[
        "Input Prompt",
        "Target Label",
        "Predicted Probability",
        "Predicted Diagnosis",
        "Predicted Explanation",
        "Tokenization Time",
        "Inference Time",
        "Input Tokens",
        "Output Tokens",
    ],
)

Block 0 preview: 
 {
  "diagnosis": "sepsis",
  "probability": 75,
  "explanation": "Suspected infection indicated by...
Decoded match: True
JSON string: {
  "diagnosis": "sepsis",
  "probability": 75,
  "explanation": "Suspected infection indicated by e...
Parsed JSON: {'diagnosis': 'sepsis', 'probability': 75, 'explanation': 'Suspected infection indicated by elevated CRP 44.5 mg/L and neutrophilia 80%; acute organ dysfunction with respiratory SOFA increase from presumed baseline 0 to 3 (PaO2/FiO2 130) at hour 5, representing ≥2 point rise.'}
Block 1 preview: 
 {
  "diagnosis": "sepsis",
  "probability": 78,
  "explanation": "Suspected infection indicated by...
Decoded match: True
JSON string: {
  "diagnosis": "sepsis",
  "probability": 78,
  "explanation": "Suspected infection indicated by e...
Parsed JSON: {'diagnosis': 'sepsis', 'probability': 78, 'explanation': 'Suspected infection indicated by elevated CRP 44.5 mg/L, neutrophilia 80%, and tachycardia (HR 102 bpm). Organ dysfuncti

In [17]:
df

,Input Prompt,Target Label,Predicted Probability,Predicted Diagnosis,Predicted Explanation,Tokenization Time,Inference Time,Input Tokens,Output Tokens
0,,1,0.75,sepsis,Suspected infection indicated by elevated CRP ...,99.7413,99.7413,3214,80
1,,1,0.78,sepsis,Suspected infection indicated by elevated CRP ...,143.4894,143.4894,3214,97
2,,1,0.75,sepsis,Suspected infection indicated by elevated CRP ...,146.6360,146.6360,3214,87
3,,1,0.78,sepsis,Patient meets Sepsis-3 criteria with suspected...,83.4366,83.4366,3214,104
4,,1,0.75,sepsis,Patient shows signs of suspected infection wit...,154.7786,154.7786,3214,97
...,...,...,...,...,...,...,...,...,...
985,,1,0.85,sepsis,Suspected infection indicated by markedly elev...,30.8925,30.8925,3214,77
986,,1,0.8,sepsis,Patient shows signs of suspected infection wit...,40.4363,40.4363,3214,86
987,,1,0.85,sepsis,Suspected infection indicated by markedly elev...,29.8703,29.8703,3214,79
988,,1,0.85,sepsis,Patient shows signs of suspected infection wit...,35.8362,35.8362,3214,93


In [18]:
df.to_csv(
    output_path,
    index=False,
)